# Reddit Climate Change - Data Preparation
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [1]:
# Loading packages
import pandas as pd
import matplotlib.pyplot as plt

## Load data and pre-processing

In [2]:
# Loading data
raw_comments = pd.read_csv('data/the-reddit-climate-change-dataset-comments.csv',nrows=1000)
raw_posts = pd.read_csv('data/the-reddit-climate-change-dataset-posts.csv',nrows= 1000)

In [3]:
# Clean Post data set from empty posts, removed posts, deleted posts
clean_posts = raw_posts.drop(raw_posts[raw_posts['selftext'] == '[removed]'].index)
clean_posts = clean_posts.drop(clean_posts[clean_posts['selftext'] == '[deleted]'].index)
clean_posts = clean_posts.drop(clean_posts[clean_posts['selftext'] == ""].index) #NaN removal does not work

In [4]:
# Drop duplicates in post and comment dataset
clean_posts = clean_posts.drop_duplicates(subset=["selftext","type"])
clean_comments = raw_comments.drop_duplicates(subset=["body","type"])

In [ ]:
# Lowercase comments and post tiltes and text

In [5]:
# Sort out the word climate change in comment and post dataset
deleted_words = ("climate","change")

for x in deleted_words:
    clean_posts["selftext"] = clean_posts["selftext"].str.replace(x, "", case=False)
    clean_posts["title"] = clean_posts["title"].str.replace(x, "", case=False)
    clean_comments["body"] = clean_comments["body"].str.replace(x, "", case=False)


/var/folders/94/kbz524cd32j80_2tm_hwnr780000gn/T/ipykernel_97033/2204490000.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_comments["body"] = clean_comments["body"].str.replace(x, "", case=False)


In [ ]:
# Sort out bot subreddits in comment dataset

In [ ]:
# Sort out bot subreddits in post dataset

In [ ]:
# Convert timestamp

In [ ]:
# Output CSV file with relevant data